In [1]:
import json
import os
import requests
import time
import yaml


In [79]:
# /Users/nxo/Workspace/GitHub/arcsaef/config/reporting.yaml
with open('config/reporting.yaml', 'r') as file:
        cf = yaml.safe_load(file)

In [84]:
# Highlight all journal articles of interest
# Run get_doi_ipop.js
# Save results as doi_org.txt
# /Users/nxo/Workspace/GitHub/arcsaef/data/all/doi_org.txt'
with open('data/all/doi_org.txt', mode= "r", encoding="utf-8") as f:
    doi_org_raw = f.readlines()

In [191]:
# Use doi a the key
# Use the values in ipop
doi_org = {}
for elem in doi_org_raw:
    if not elem.startswith("#"):
        if len(elem.split(":") ) == 2:
            doi_org[elem.split(":")[0].strip()] = elem.split(":")[1].strip()

In [192]:
# load organisation in config file
nrpt_orgs = json.loads(cf['non-report']['organisations'])
rpt_orgs  = json.loads(cf['report']['organisations'])
nrpt_orgs.update(rpt_orgs)
nrpt_orgs_uc = {}

for e in nrpt_orgs:
    nrpt_orgs_uc[e.upper()] = nrpt_orgs.get(e)

# update organisation abbreviation with it's long name
for o in doi_org:
    chopped = doi_org.get(o).split(";")
    if( len(chopped) == 1 ):
        for x in nrpt_orgs_uc:
            if x == doi_org.get(o).upper():
                doi_org[o] = nrpt_orgs.get(x)
# handle output with >1 ipop
    else:
        org_split = []
        for chop in chopped:
            if chop.strip().upper() in nrpt_orgs_uc.keys():
                org_split.append(nrpt_orgs_uc.get(chop.strip().upper()))
        doi_org[o] = org_split

In [11]:
url = "https://api.openalex.org/works/https://doi.org/"

for doi in doi_org:
    # there is no SAEF id that can be used to retrieve research output attributed to SAEF
    # therefore it is safer to use DOI's of known (recorded in Zotero) SAEF output 
    # /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/
    response = requests.get(f"{url}{doi}") 

    if response.status_code == 200:
        page  = json.loads(response.content) # dictionary 
        fname = f"{'data/jnls_202408/'}{page.get('id').split('/')[3]}.json"

        with open(fname, mode= "w", encoding="utf-8") as f:
            json.dump(page, f)

    time.sleep(1) # sleep for 1 second satisfies max 10 call per sec limit


In [220]:
# /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/W4392715378.json
with open('data/jnls_202408/W4392715378.json', mode= "r", encoding="utf-8") as f:
    work = json.load(f)

In [262]:
# /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/
for file in os.listdir('data/jnls_202408/'):
    with open(f"{'data/jnls_202408/'}{file}", mode= "r", encoding="utf-8") as f:
        work = json.load(f)

        collab_orgs  = {}
        institutions = []

        if work.get('doi')[16: ] in doi_org.keys():
            for i in work.get('authorships')[0]['institutions']:
                institutions.append(i['display_name'])

                if work.get('doi')[16: ] in collab_orgs.keys():
                    collab_orgs[ work.get('doi')[16: ] ].append(institutions)
                else:
                    collab_orgs[ work.get('doi')[16: ] ] = institutions

        # for d in doi_org:
        #     if d == work.get('doi')[16: ]:
        #         for author in work.get('authorships'):
        #             print(author)
                #     if author.get('institutions')[0]['display_name'] != doi_org.get(d):
                #         institutions.append(author.get('institutions')[0]['display_name'])
                # if doi_org.get(d) in collab_orgs.keys():
                #     collab_orgs[doi_org.get(d)].append(institutions)
                # else:
                #     collab_orgs[doi_org.get(d)] = institutions

In [263]:
collab_orgs

{}

In [196]:
# /Users/nxo/Workspace/GitHub/arcsaef/data/jnls_202408/
for file in os.listdir('data/jnls_202408/'):
    with open(f"{'data/jnls_202408/'}{file}", mode= "r", encoding="utf-8") as f:
        work = json.load(f)
        print(doi_org.get(work.get('doi')[16: ]))

['University of Wollongong', 'Monash University']
['La Trobe University', 'Monash University']
Monash University
Monash University
Monash University
University of Wollongong
Monash University
University of Wollongong
Monash University
University of Wollongong
['Monash University', 'King Juan Carlos University']
['James Cook University', 'Western Australia Museum']
['Monash University', 'University of Wollongong']
Monash University
Monash University
['Monash University', 'Australian Antarctic Division']
University of Wollongong
University of Wollongong
South Australia Museum
Monash University
University of Wollongong
['Monash University', 'Berkeley Geochronology Center']
['University of Wollongong', 'University of the Balearic Islands']
James Cook University
University of Wollongong
La Trobe University
Monash University
Monash University
['Queensland University of Technology', 'University of Wollongong']
La Trobe University
Monash University
University of Wollongong
None
None
Monash Uni

In [64]:
for x in doi_org:
    d = f"{'https://doi.org/'}{x}"
    if d == work['ids']['doi']:
        print( doi_org.get(x) )
        for author in work.get('authorships'):
            print(author.get('institutions')[0]['display_name'])

QUT
Queensland University of Technology
Queensland University of Technology
Queensland University of Technology
Queensland University of Technology
CSIRO Land and Water
Queensland University of Technology
Deakin University
Queensland University of Technology
